In [4]:
from duckduckgo_search import DDGS
from fastcore.all import *
from fastdownload import download_url
from fastai.vision.all import *
from time import sleep


## What's this doing?
Downloading all of the required library for the project chunck
- **duckduckgo_search:** for searching for images
- **fastcore.all:** for the fastcore library

In [13]:
def search_images(term, max_images=5):
    print(f'Searching for: {term}')
    with DDGS() as ddgs:
        results = ddgs.images(keywords=term, max_results=max_images)
        return L(results).itemgot('image')

## What's this doing?
creating a function that returns at most 5 images from duck duck go. Something to note:
- ddg_images has a lot of optional arguments, which you can find here: https://help.duckduckgo.com/settings/params/
- L(SOME_SEARCH_RESULT).itemgot('image') makes getting the list of images from ddg so much easier as a dictionare relating to reach search is what ddg_images produces

In [14]:
search_terms = ['Oak Leaves', 'Maple Leaves']
path = Path('maple_or_oak')
for search_term in search_terms:
    folder = ''.join(search_term.split())
    dest = (path/folder)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(term=f'{search_term} photos', max_images=30))
    resize_images(path/folder, max_size=400, dest=path/folder)
    failed = verify_images(get_image_files(path))
    failed.map(Path.unlink)
    len(failed)
    sleep(10)

Searching for: Donald Trump photos
Searching for: Angry Orange photos


## What's this doing?
for identified search terms, in this case 'The Scarlet Letter' and 'Huckleberry Finn', it downloads related book cover images to folderes defined by the key search term, and then removes files that aren't images

In [ ]:
path = Path('donald_or_not')
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=57),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=8)


In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

In [ ]:
download_url(search_images('donald trump beach photo', max_images=1)[0], 'trump.jpg')
Image.open('trump.jpg').to_thumb(256,256)

In [ ]:
is_trump, what, probs = learn.predict(PILImage.create('trump.jpg'))
print(f"This is a: {is_trump}.")
print(f"Probability it's {is_trump}: {probs[0]:.4f}")
print('what is this?', what)
